In [1]:
import pandas as pd
import json
import json_lines
import numpy as np
import re
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences

np.random.seed(seed=0)

In [2]:
news_id = []
article_original = []
abstractive = []
extractive = []
with open('./data/extract_news/train.jsonl', 'rb') as f: # opening file in binary(rb) mode    
   for item in json_lines.reader(f):
#        print(item) #or use print(item['X']) for printing specific data
        news_id.append(item['id'])
        article_original.append(','.join(item['article_original']).replace(',', ' '))
        abstractive.append(item['abstractive'])
        extractive.append(item['extractive'])

In [3]:
data = pd.DataFrame({'news_id':news_id, 'article_original': article_original, 'abstractive': abstractive, 'extractive': extractive})
data.head()

,news_id,article_original,abstractive,extractive
0,327827480,당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것으...,지난 6일 당진시의회 행정사무감사에서 '합창단이 소리를 작게 낼 것이니 알고 있으라...,"[0, 1, 2]"
1,339840364,미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있다...,LA 에인절스의 최지만이 맹활약을 하여 시즌 타율 0.250에서 0.313으로 올리...,"[0, 1, 3]"
2,371071597,인천 부영공원 운영 생활야구협회 80여 동호회 팀에 260만 원씩 받아 국유지로 임...,16일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체육야구협회가 ...,"[6, 7, 10]"
3,354806783,대구·경북첨단의료산업진흥재단 의약생산센터는 항암주사제 무균충전 시설을 갖추고 있다....,대구·경북첨단의료산업진흥재단 의약생산센터는 약사법 시행규칙에서 정한 바에 따라 전용...,"[1, 6, 4]"
4,347022773,국내 유통되는 탄산음료 중 식품의약품안전처에 품질인증을 받은 제품이 하나도 없는 것...,식품의약품안전처는 29일 어린이가 즐겨마시는 음료를 대상으로 영양성분을 조사한 결과...,"[2, 3, 10]"


In [4]:
# article_original과 abstractive 에서 한글과 공백 제외하고 정규식으로 삭제
data['article_original'] = data['article_original'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
data['abstractive'] = data['abstractive'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
data.head()

,news_id,article_original,abstractive,extractive
0,327827480,당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것으...,지난 일 당진시의회 행정사무감사에서 합창단이 소리를 작게 낼 것이니 알고 있으라라는...,"[0, 1, 2]"
1,339840364,미국 메이저리그에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있다 에인절...,에인절스의 최지만이 맹활약을 하여 시즌 타율 에서 으로 올리며 주전 입지를 굳혀가...,"[0, 1, 3]"
2,371071597,인천 부영공원 운영 생활야구협회 여 동호회 팀에 만 원씩 받아 국유지로 임대료 없고...,일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체육야구협회가 무상...,"[6, 7, 10]"
3,354806783,대구경북첨단의료산업진흥재단 의약생산센터는 항암주사제 무균충전 시설을 갖추고 있다 대...,대구경북첨단의료산업진흥재단 의약생산센터는 약사법 시행규칙에서 정한 바에 따라 전용의...,"[1, 6, 4]"
4,347022773,국내 유통되는 탄산음료 중 식품의약품안전처에 품질인증을 받은 제품이 하나도 없는 것...,식품의약품안전처는 일 어린이가 즐겨마시는 음료를 대상으로 영양성분을 조사한 결과 조...,"[2, 3, 10]"


In [5]:
# 불용어 대략 600개 txt 파일 
SW = set()
with open('./data/stopwords.txt', 'r', encoding='UTF8') as f:
    for word in f:
        SW.add(word.replace('\n', ''))
print(len(SW))

595


In [24]:
# 불용어100개 txt 파일 
SW = set()
with open('./data/stopword100.txt', 'rb') as f:
    for word in f:
        SW.add(word.replace('\n', ''))
len(SW)

100

In [6]:
# 불용어 처리
# original 기사는 불용어 처리 O, Summary는 불용어 처리 X
from konlpy.tag import Okt
okt = Okt()
# stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

tokens = []
for sentence in data['article_original']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in SW] # 불용어 제거
    tokens.append(temp_X)
data['article_original'] = tokens

data.head()

KeyboardInterrupt: 

In [52]:
tokens2 = []
for sentence in data['abstractive']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in SW] # 불용어 제거
    tokens2.append(temp_X)
data['abstractive'] = tokens2
data.head()

,news_id,article_original,abstractive,extractive
0,327827480,당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것으...,지난 6일 당진시의회 행정사무감사에서 '합창단이 소리를 작게 낼 것이니 알고 있으라...,"[0, 1, 2]"
1,339840364,미국 메이저리그(MLB)에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있다...,LA 에인절스의 최지만이 맹활약을 하여 시즌 타율 0.250에서 0.313으로 올리...,"[0, 1, 3]"
2,371071597,인천 부영공원 운영 생활야구협회 80여 동호회 팀에 260만 원씩 받아 국유지로 임...,16일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체육야구협회가 ...,"[6, 7, 10]"
3,354806783,대구·경북첨단의료산업진흥재단 의약생산센터는 항암주사제 무균충전 시설을 갖추고 있다....,대구·경북첨단의료산업진흥재단 의약생산센터는 약사법 시행규칙에서 정한 바에 따라 전용...,"[1, 6, 4]"
4,347022773,국내 유통되는 탄산음료 중 식품의약품안전처에 품질인증을 받은 제품이 하나도 없는 것...,식품의약품안전처는 29일 어린이가 즐겨마시는 음료를 대상으로 영양성분을 조사한 결과...,"[2, 3, 10]"


-----------
    seq2seq 훈련을 위해서는 디코더의 입력과 레이블에 시작 토큰과 종료 토큰을 추가한다.
    시작 토큰은 'sostoken', 종료 토큰은 'eostoken'이라 명명하고 앞, 뒤로 추가

In [61]:
data['decoder_input'] = data['abstractive'].apply(lambda x : 'sostoken '+ x)
data['decoder_target'] = data['abstractive'].apply(lambda x : x + ' eostoken')
data.head()

,news_id,article_original,abstractive,extractive,decoder_input,decoder_target
0,327827480,당진시 문화관광과를 대상으로 하는 행정사무감사에서 당진시립합창단 관계자가 보낸 것으...,지난 일 당진시의회 행정사무감사에서 합창단이 소리를 작게 낼 것이니 알고 있으라라는...,"[0, 1, 2]",sostoken 지난 일 당진시의회 행정사무감사에서 합창단이 소리를 작게 낼 것이니...,지난 일 당진시의회 행정사무감사에서 합창단이 소리를 작게 낼 것이니 알고 있으라라는...
1,339840364,미국 메이저리그에서 활동하는 한국 선수들의 시즌 초반 희비가 엇갈리고 있다 에인절...,에인절스의 최지만이 맹활약을 하여 시즌 타율 에서 으로 올리며 주전 입지를 굳혀가...,"[0, 1, 3]",sostoken 에인절스의 최지만이 맹활약을 하여 시즌 타율 에서 으로 올리며 주...,에인절스의 최지만이 맹활약을 하여 시즌 타율 에서 으로 올리며 주전 입지를 굳혀가...
2,371071597,인천 부영공원 운영 생활야구협회 여 동호회 팀에 만 원씩 받아 국유지로 임대료 없고...,일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체육야구협회가 무상...,"[6, 7, 10]",sostoken 일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체...,일 부평구와 협회 등에 따르면 부영공원 안에 있는 야구장을 구생활체육야구협회가 무상...
3,354806783,대구경북첨단의료산업진흥재단 의약생산센터는 항암주사제 무균충전 시설을 갖추고 있다 대...,대구경북첨단의료산업진흥재단 의약생산센터는 약사법 시행규칙에서 정한 바에 따라 전용의...,"[1, 6, 4]",sostoken 대구경북첨단의료산업진흥재단 의약생산센터는 약사법 시행규칙에서 정한 ...,대구경북첨단의료산업진흥재단 의약생산센터는 약사법 시행규칙에서 정한 바에 따라 전용의...
4,347022773,국내 유통되는 탄산음료 중 식품의약품안전처에 품질인증을 받은 제품이 하나도 없는 것...,식품의약품안전처는 일 어린이가 즐겨마시는 음료를 대상으로 영양성분을 조사한 결과 조...,"[2, 3, 10]",sostoken 식품의약품안전처는 일 어린이가 즐겨마시는 음료를 대상으로 영양성분을...,식품의약품안전처는 일 어린이가 즐겨마시는 음료를 대상으로 영양성분을 조사한 결과 조...


In [63]:
# 인코더 입력,디코더 입력, 레이블
encoder_input = np.array(data['article_original'])
decoder_input = np.array(data['decoder_input'])
decoder_target = np.array(data['decoder_target'])

-------------
### 정수 인코딩
- 단어에 정수를 부여하는 방법 중 하나로 단어를 빈도수 순으로 정렬한 단어 집합(vocabulary)을 만들고, 빈도수가 높은 순서대로 차례로 낮은 숫자부터 정수를 부여하는 방법
    
- 이제 기계가 텍스트를 숫자로 처리할 수 있도록 훈련 데이터와 테스트 데이터에 정수 인코딩을 수행 
- 훈련 데이터에 대해서 단어 집합(vocaburary)을 만듦
- 우선, 원문에 해당되는 encoder_input 에 대해서 수행

In [65]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [67]:
src_tokenizer = Tokenizer()
src_tokenizer.fit_on_texts(encoder_input)

# 단어 집합이 생성되는 동시에 각 단어에 고유한 정수가 부여되었습니다. word_index로 확인
# print(src_tokenizer.word_index)

In [70]:
len(src_tokenizer.word_index)

960812

In [ ]:
# 등장 빈도수가 3회 미만인 단어들이 이 데이터에서 얼만큼의 비중을 차지하는지 확인하기
threshold = 3
total_cnt = len(src_tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in src_tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
src_vocab = 10000 # 위에서 나온 결과를 고려해서 단어의 집합 크기를 결정
src_tokenizer = Tokenizer(num_words = src_vocab) 
src_tokenizer.fit_on_texts(encoder_input)

# 텍스트 시퀀스를 정수 시퀀스로 변환
encoder_input_train = src_tokenizer.texts_to_sequences(encoder_input)
# encoder_input_test = src_tokenizer.texts_to_sequences(encoder_input_test)

In [ ]:
# 전체 단어 개수 중 빈도수 3이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

In [ ]:
src_tokenizer = Tokenizer(num_words = vocab_size) 
src_tokenizer.fit_on_texts(encoder_input)

# 텍스트 시퀀스를 정수 시퀀스로 변환
encoder_input_train = src_tokenizer.texts_to_sequences(encoder_input) 
encoder_input_test = src_tokenizer.texts_to_sequences(encoder_input_test)

In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

### 빈 샘플 제거

### 패딩

### seq2seq + attention으로 요약 모델 설계 및 훈련시키기

1. 데이터 전처리 과정
    - 영어, 숫자, 특수문자 제거
    - 단어 정규화, 형태소 추출(stemming?), 불용어 처리, 단어길이 제한 -> 토큰화
    - 
2. 